<a href="https://colab.research.google.com/github/gachet/ad-1-24/blob/main/eda/EDA_%2B_regression_Electric_Motor_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F236410%2F504254%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241003%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241003T175227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D47b03894044b1d63ebcd043c0830be1215a6516fa3fc2df275e90a9f631c916e28f00f8d92a2cf12dbbf3f0b293b2cd31c6cc67fb2cd75560138b4d3f8a39335a6d5ad75ca94c2d54a33a82f2ea299f90c40e4a33f866504a3109b31d8b4e0b4759597838dd4d7eaf550561de4610a7be970a3e8fd44bf4f88f04a133fca59905711f7d6652cdf7bf486d7c41dd9f70e01330499c1f85e706962636c1fd65ad7fbd77044e57592a36740b23f9a99b7a734405cd71ef886c9b88d6e8af53334f74de56007ff8bab722df0e6429499051c0589a5af72d421a03527ff39a65ecbb854a2ddb23de84c6eddcae0ac20ec66b5bc820616ba15dd1a776a5a1a9ae1cb58'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install tensorflow-gpu==2.0.0-alpha

In [ ]:
#
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import os

print(os.listdir("../input"))

In [ ]:
data = pd.read_csv("../input/pmsm_temperature_data.csv")
data = data.drop("profile_id", axis=1)

In [ ]:
data.head(5)

In [ ]:
data.hist(figsize = (20,20))
plt.show()

In [ ]:
column = ['coolant', 'motor_speed', 'torque']

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(5,20))
fig.subplots_adjust(hspace=0.2)

for (ax, i) in zip(axes, column):
    sns.violinplot(x=data[i], ax=ax)
    plt.plot()

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(5,20))
fig.subplots_adjust(hspace=0.2)

for (ax, i) in zip(axes, column):
    sns.distplot(data[i], ax=ax)
    plt.plot()

In [ ]:
x = data.drop("motor_speed", axis=1).values
y = data["motor_speed"].values.reshape(-1, 1)

x = tf.cast(x, dtype=tf.float32)
y = tf.cast(y, dtype=tf.float32)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(128)

In [ ]:
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(0.001)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='sigmoid', input_shape=(11,)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='tanh')
])

In [ ]:
loss_metric = tf.keras.metrics.Mean(name='loss')

In [ ]:
def train_step(data, labels):
    with tf.GradientTape() as tape:

        output = model(data, training=True)

        loss = loss_fn(labels, output)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    loss_metric.update_state(loss)

In [ ]:
loss_history = []

for epoch in range(1, 11):

    loss_metric.reset_states()

    for (data, labels) in dataset:
        train_step(data, labels)

    mean_loss = loss_metric.result()

    loss_history.append(mean_loss)

    print('Epoch: {}  MSE: {:.3f}'.format(epoch, mean_loss))


In [ ]:
h_loss = np.array(loss_history)

In [ ]:
plt.plot(h_loss)
plt.title("loss")
plt.show()